# Trabalho Prático 1

## Introdução

A resolução deste trabalho prático tem como objetivo aprofundar o conhecimento na implementação de sessões cifradas e seguras num ambiente entre um emissor e um recetor.
Os objetivos principais deste trabalho prático são:

- A construção de uma sessão de comunicação síncrona entre um emissor e um recetor.
- A utilização do protocolo de **Diffie_Hellman** com verificação de chave e autenticação dos agentes na sessão anteriormente referida.
- Utilização do **DSA** para a autenticação dos agentes.
- Utilização do **TAES** como cifra simétrica, com autenticação do criptograma em cada superbloco.
- Finalmente, o último objetivo passa por implementar novamente o cenário anterior mas, desta feita, utilizando o **ECDH** (Elliptic Curve Diffie-Hellman) em vez do **DH** e o **ECDSA** (Elliptic Curve Digital Signature Algorithm) em vez do **DSA**.

Este relatório está escrito de forma a que o texto escrito entre o código desenvolvido seja suficientemente explicativo do que está a ser implementado, o que, na nossa opinião, permite uma leitura e compreensão facilitada do mesmo.

## Sessão Síncrona com _Diffie-Hellman_

### Imports

Esta secção tem como objetivo importar os módulos **Python** necessários para desenvolver a sessão síncrona entre os agentes, utilizando o protocolo **DH** e o **DSA**.

In [1]:
import os,io
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.exceptions import *
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição de funções auxiliares 

Esta secção tem como objetivo definir funções auxiliares, tais como a função que gera o **hash** de uma mensagem, de **cifra** e de **decifra**, que serão posteriormente utilizados pelos agentes que participam na sessão síncrona

In [2]:
def Hash(s):
    digest = hashes.Hash(hashes.SHA256(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

def next_tweak(initial, current_counter):
    str = format(current_counter, '063b')
    str = str + '0'
    b_str = bytes(str,'utf-8')
    return (initial + b_str)

# Algo está errado numa destas operações!
# A cifragem e decifragem não está a dar o mesmo resultado!
# Função bytes_xor não a conheces. poderá ser daí?
# Ou a operação de decifra deve ter algo de diferente em relação à operação de cifra?

def bytes_xor(a, b) :
    return bytes(x ^ y for x, y in zip(a, b))

def cifra(key,block,tweak,cipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,tweak)
    e_s_x_hw = cipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram

def decifra(key,block,tweak,decipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,tweak)
    e_s_x_hw = decipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram


# Falta pôr funções de cifra e de decifra
    

### Definição da classe de multiprocessamento

Esta secção tem o propósito de definir a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [3]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def auto(self,proc=None):
        if proc == None:
            self.left_process.start()
            self.right_process.start()
            self.left_process.join(self.timeout)
            self.right_process.join(self.timeout)
        else:
            proc.start()
            proc.join()
    
    def manual(self): 
        self.left()
        self.right()

### Geração dos parâmetros _Diffie-Hellman_

Esta secção tem o objetivo de gerar os parâmetros necessários para a geração de chaves **DH** e **DSA** por parte de ambos os agentes.

In [4]:
parameters_dh = dh.generate_parameters(generator=2,key_size=1024,backend=default_backend())
# O tamanho tem que ser 3072 bits, alterar depois.

### Geração das chaves permanentes dos agentes

Esta secção tem como objetivo gerar as chaves permanentes dos agentes envolvidos na comunicação, tendo em conta que estas são as que são utilizadas na troca de chaves utilizando o protocolo **Diffie-Hellman** e na assinatura digital de mensagens.

In [5]:
# Geração das chaves do Emissor
emitter_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
emitter_dsa_pk = emitter_dsa_sk.public_key() #chave pública DSA

# Geração das chaves do Recetor
receiver_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
receiver_dsa_pk = receiver_dsa_sk.public_key() #chave pública DSA

### Definição do Emissor e do Recetor

Nesta secção está definido o comportamento de cada um dos agentes ao utilizar esta sessão síncrona segura de troca de informação. Na definição destes dois agentes, é assumido que as chaves públicas **DSA** utilizadas na autenticação dos agentes já são conhecidas por ambos. As chaves **DH** são geradas por sessão de modo a acordar uma chave temporária para que seja possível cifrar e decifrar uma mensagem utilizando uma primitiva simétrica.

**PROCESSO DE CIFRA E DECIFRA NÃO ESTÁ CORRECTO! NÃO PRODUZ OS MESMOS RESULTADOS**

In [6]:
def Emissor(connection):
    #Implementação Protocolo Diffie-Hellman
    emitter_dh_sk = parameters_dh.generate_private_key()
    emitter_dh_pk = emitter_dh_sk.public_key()
    pub = emitter_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = emitter_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk': pub, 'sig': signature}
    connection.send(message)
    receiver_first = connection.recv()
    receiver_dh_public_key = serialization.load_pem_public_key(receiver_first['pk'],default_backend())
    receiver_dsa_pk.verify(receiver_first['sig'],receiver_first['pk'],hashes.SHA256())
    master_key = emitter_dh_sk.exchange(receiver_dh_public_key)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    message_size = 2**10
    gen_bytes = os.urandom(message_size)
    inputs = io.BytesIO(gen_bytes)
    #inicialização do tweak como um nounce de 64 bits (n/2)
    tweak_init = os.urandom(8)
    
    connection.send(tweak_init)
    
    buffer = bytearray(16)
    cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).encryptor()
    i = 1
    try:
        while inputs.readinto(buffer):
            # Falta ver a parte de autenticação dos superblocos do criptograma
            current_tweak = next_tweak(tweak_init,i)
            cryptogram = cifra(shared_key,bytes(buffer),current_tweak,cipher)
            connection.send(cryptogram)
            i += 1
        connection.send(cipher.finalize())
    except Exception:
        print('Erro no emissor!')
    connection.send('finalized')   
    inputs.close()
    connection.close()    

In [7]:
def Recetor(connection):
    #Implementação protocolo Diffie-Hellman
    emitter_first = connection.recv()
    receiver_dh_sk = parameters_dh.generate_private_key()
    receiver_dh_pk = receiver_dh_sk.public_key()
    emitter_dh_public_key = serialization.load_pem_public_key(emitter_first['pk'],default_backend())
    emitter_dsa_pk.verify(emitter_first['sig'],emitter_first['pk'],hashes.SHA256())
    master_key = receiver_dh_sk.exchange(emitter_dh_public_key)
    pub = receiver_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = receiver_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk':pub,'sig':signature}
    connection.send(message)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    outputs = io.BytesIO()
    tweak_init = connection.recv()
    
    cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).decryptor()
    i = 1
    try:
        while True:
            buffer = connection.recv()
            if(buffer == 'finalized'):
                outputs.write(cipher.finalize())
                break
            else:
                #print(buffer)
                cryptogram = buffer
                current_tweak = next_tweak(tweak_init,i)
                text = decifra(shared_key,cryptogram,current_tweak,cipher)
                i += 1
                outputs.write(text)
        print(outputs.getvalue())
    except Exception as Err:
        print('Erro no recetor! {0}'.format(Err))
        
    outputs.close()
    connection.close()
    

In [8]:
BiConnection(Emissor,Recetor).auto()

CRYPT:
DECRYPT
1b'\xc8\x90\xd6y\xcd\xb9\xe1\x82\xc88\xd9\x9dz\xc4\xc6\xd3'
1b'u+l\xeb\xe5\x94\xb2\x85\xde\xb7w\x1f\x14mu\n'
2b'\xad\xcf\xa1+\xeb\xa8e\xa4v(\xf9\x04\xf4\xd1\x99S'
3b'\xaaQ\xd8\xed\x9e\xc5E#\tH\x85\x91\x9b/\x08t'
2b'1`\xb7\x05\xa0hr}\x9e\x9d\xbeA\x0b\x01H\x1a'
4b'Y\x8b\xac2^\xd91\xe3~\xed=\xc4h`\xcax'
5b'lX\xa6\x88;\x07\xaa\xd4`Z\xe9$\x8ap\xe4\r'
6b'4QjD\xba\xea-\x9c\xc7\x8dg\x13G\xacU\xb4'
7b'\xd7z$\xa6C\x8a\xbd\xe7+K.\xd3\xd0\xac\n1'
3b'\x14\xa0\x8ff\xc3\x87\x19\xe0(5_\xd7r\xb2\xdc\x05'
8b'\xf2\xc5I\x07^\xf7\xe5K:1\xaa\x1c\x08\x8f\xe86'
4b'\xed5P"%\xcc\xf7\x91\x06\xedY\xd5\xf9\xd0E\x13'
9b'i\x08t\xad\x7f_\x9dyh\xa0\x8b\x96O\xec\xbf\xae'
5b'\xd2\xdd\x1e\x0c\xd9\x7f#\xea\x9b\xd1\x14]\xa8\x91[\xca'
10b'\xbf\xef\x1c~\xb6\xaaX>\x08O\x16\xe5\xf6\x98\x0b\xe8'
11b'\x87Hy:\xec\xe9\xbdQH\xb3\xb6T\xdb\x87\xdd\xe6'
12b'Irz\tyA=\xfa\x1f\x19\x0c\x0e(\xec\xf4\xf8'
13b'T\xa5<\xfb\xd8\xb2\x8b\x14\x00\xfa\x0e\xa2\x8b\x1b^u'
14b'\xf0gM\x00\xff\xd4(\x92_\x8aa\xb0S<\tB'
15b'\x14\x11\xb00\xa